# VacationPy

In [12]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
#creating path to output data csv
weatherInfoFile = "../WeatherPy/output_data/cities.csv"

#reading in csv
uglyWeather = pd.read_csv(weatherInfoFile)

#deletes the first column named unnamed, which counted from 0-the end of the datset for no reason. 
weather = uglyWeather.drop(columns=["Unnamed: 0"])
weather.head()

,City,Cloudiness,Country,Date,Humidity,Lat.,Lng.,Max Temp,Wind Speed
0,Port Elizabeth,20,ZA,1579983852,93,-33.92,25.57,66.20,4.70
1,Ndola,25,ZM,1579983852,95,-12.96,28.64,63.77,4.34
2,Salalah,56,OM,1579983852,83,17.02,54.09,73.00,5.84
3,Lata,0,PT,1579983853,92,40.16,-8.33,53.01,1.16
4,Pevek,100,RU,1579983732,96,69.70,170.31,-16.02,3.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
#Creating coordinates and humidity lists which will be added to the heatmap
coordinates = weather[['Lat.','Lng.']]
humidity = weather['Humidity']

#Map center will focus the figure below on Kriti, in the mediterranean
#sea to give the map a nice focal point
mapCenter = (35.2401, 24.8093)

# Plot Heatmap, setting center of map, and zoom level to 2 to get rid of 
#multiple maps in same fig
fig = gmaps.figure(center=mapCenter,zoom_level=2)

#heat map with weight set to humidity 
heatmap_layer = gmaps.heatmap_layer(coordinates, weights=humidity,
                                    max_intensity=100,point_radius = 5)

#adding the heatmap layer to the figure
fig.add_layer(heatmap_layer)

#display map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
#Drop country, date,lat, and long values to focus on weather data and city names 
updated_df = pd.DataFrame(weather, columns = ["City","Cloudiness","Max Temp", "Wind Speed", "Humidity"])

#Now I get to be picky and set the values of the weather that I enjoy most
#Temperature should be 65-80 F
#Wind should be minimal, no more than 10 mph
#Clear sky is better than cloudy sky, I set the max cloudiness value to 20%
#I dislike humidity so I set te value for humidity at less than 50%
#Drop all rows that do not fit this criteria

idealCloud = updated_df["Cloudiness"] <=20
idealTemp = ((updated_df["Max Temp"] <=80) & (updated_df["Max Temp"] > 65))
idealWind = updated_df["Wind Speed"] < 10
idealHumidity = updated_df["Humidity"] < 50

#narrowing table down to show preferred conditions, creating new dataframe from that table
idealCities = updated_df[idealCloud & idealTemp & idealWind & idealHumidity]
narrowed_city_df = pd.DataFrame(idealCities)
narrowed_city_df 

,City,Cloudiness,Max Temp,Wind Speed,Humidity
130,Guerrero Negro,0,71.26,4.36,43
134,Leeton,0,73.85,1.43,31
142,Alice Springs,0,71.15,7.87,49
167,Lompoc,1,72.00,3.36,48
352,Marabu,2,75.00,4.59,10
371,Man,1,74.93,2.28,16
418,Seguela,0,74.84,1.63,14
443,Sanford,0,71.01,8.90,22


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
#creating a dataframe which is strictly geographical with no weather information
hotel_df = pd.DataFrame(weather, columns = ["City" , "Country", "Lat.", "Lng."])

#creating empty column to hold hotel names later and merging with ideal vacation spots
hotel_df["Hotel Name"] = ''
hotel_df = hotel_df.merge(narrowed_city_df)
hotel_df

,City,Country,Lat.,Lng.,Hotel Name,Cloudiness,Max Temp,Wind Speed,Humidity
0,Guerrero Negro,MX,27.98,-114.06,,0,71.26,4.36,43
1,Leeton,AU,-34.57,146.40,,0,73.85,1.43,31
2,Alice Springs,AU,-23.70,133.88,,0,71.15,7.87,49
3,Lompoc,US,34.64,-120.46,,1,72.00,3.36,48
4,Marabu,MY,4.41,114.01,,2,75.00,4.59,10
5,Man,CI,7.41,-7.55,,1,74.93,2.28,16
6,Seguela,CI,7.96,-6.67,,0,74.84,1.63,14
7,Sanford,US,28.80,-81.27,,0,71.01,8.90,22


# Calling the Google API

In [17]:
#empty list to hold hotel info later
hotels = []

#counter variable
counter = 0

#for loop which will loop through cities in the ideal weather cities 
for cities in hotel_df['City']:
    #the target variables set what the parameters will be, as the loop is iterated
    target_coordinates = str(hotel_df['Lat.'][counter])+' , '+str(hotel_df['Lng.'][counter])
    target_search = "hotel"
    target_radius = 5000
    target_type = "lodging"
    counter +=1
    #parameters get passed to the url when calling the api to define the search
    params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key}
    
    #base url for the api which we will be calling
    test_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #try except statement with requests embedded, this will prevent errors when trying to fill the hotel name list
    #due to missing information
    try:  
        response = requests.get(test_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)
        
#appending dataframe with hotel names
hotel_df['Hotel Name'] = hotels

In [18]:
#cleaning up the dataframe
clean_hotels = hotel_df.dropna()

#removing columns related to weather information
clean_hotels = clean_hotels.drop(columns = ["Cloudiness", "Max Temp", 'Wind Speed','Humidity'])

#final clean hotels dataframe for my perfect vacation
clean_hotels

,City,Country,Lat.,Lng.,Hotel Name
0,Guerrero Negro,MX,27.98,-114.06,The Halfway Inn
1,Leeton,AU,-34.57,146.40,Kindred Studio Apartments
2,Alice Springs,AU,-23.70,133.88,DoubleTree by Hilton Hotel Alice Springs
3,Lompoc,US,34.64,-120.46,Embassy Suites by Hilton Lompoc Central Coast
4,Marabu,MY,4.41,114.01,Pullman Miri Waterfront
5,Man,CI,7.41,-7.55,Sodetherm Hotel les Cascades Ma
6,Seguela,CI,7.96,-6.67,Hotel Carrefour
7,Sanford,US,28.80,-81.27,Best Western Plus Sanford Airport/Lake Mary Hotel


In [19]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in clean_hotels.iterrows()]
locations = clean_hotels[["Lat.", "Lng."]]

In [20]:
#Map center will focus the figure below on Kriti, in the mediterranean
#sea to give the map a nice focal point
mapCenter = (35.2401, 24.8093)

# Plot Heatmap, setting center of map, and zoom level to 2 to get rid of 
# multiple maps in same fig
fig = gmaps.figure(center=mapCenter,zoom_level=2.0)
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))